In [2]:
import random
import sys

import data
from objects import *

In [3]:
def create_world():
    """Return a dict holding all installed Places."""
    w: Dict[str, Place] = {}
    for p in data.places:
        w[p] = Place(p)
        w[p].install()

    return w


def create_things(world):
    for t in data.things:
        Thing(t['name'], world[t['place']]).install()

    for m in data.mobile_things:
        Mobile_Thing(m['name'], world[m['place']]).install()


def create_exits(world):
    # TODO modify Exit to be more parallel to Weapon and Thing??
    """Install an Exit between two Places."""
    for e in data.exits:
        Exit(world[e['origin']], e['direction1'], world[e['destination']]).install()
        Exit(world[e['destination']], e['direction2'], world[e['origin']]).install()


def create_weapons(world):
    for w in data.weapons:
        Weapon(w['name'], random.choice(list(world.values())), w['damage']).install()


def create_people(world):
    names = data.names
    vamp = random.choice(names)
    names.remove(vamp)
    Vampire(vamp, random.choice(list(world.values())), False).install()
    for n in names:
        Autonomous_Person(n, random.choice(list(world.values())), random.randint(0, 3), random.randint(0, 3)).install()



In [4]:
def setup():
#     global clock, world
    clock.reset()
    clock.add_callback(Callback("tick-printer", clock, "print_tick"))

    world = create_world()
    create_things(world)
    create_exits(world)
    # create_weapons(world)
    create_people(world)

    print('The Adventures of Buffer the Vampire Slayer')
    name = input('player name: ')
    player = Avatar(name, random.choice(list(world.values())))
    player.install()

    return world, player

In [ ]:
clock = Clock()
# world, player = setup()

In [ ]:

while True:
    action = input('What would you like to do?')
    if action == 'q':
        break
    if action in ['up', 'down', 'north', 'south', 'east', 'west']:
        player.go(action)
    if action == 'look':
        player.look()
    else:
        pass

In [ ]:
clock

In [ ]:
setup()

In [5]:
n = Named_Object('test1')
o = Named_Object('test2')
p = Named_Object('test3')

In [6]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '__jedi_version__', '_i', '_ii', '_iii', '_i1', 'random', 'sys', 'data', '_i2', 'Callable', 'Dict', 'List', 'Optional', 'TypeVar', 'Union', 'TYPE_CHECKING', 'names', 'thingfind', 'find_all', 'find_exit', 'random_exit', 'current_time', 'run_clock', 'Clock', 'Named_Object', 'Container', 'Thing', 'Mobile_Thing', 'Place', 'Exit', 'Person', 'Autonomous_Person', 'Body', 'Vampire', 'Avatar', 'clock', '_i3', 'create_world', 'create_things', 'create_exits', 'create_weapons', 'create_people', '_i4', 'setup', '_i5', 'n', 'o', 'p', '_i6'])

In [ ]:
world = create_world()

In [7]:
n.install()

test1 installed!


In [11]:
cb_list = [n.install, o.install, p.install]
for x in cb_list:
    print(n == x.__self__)

True
False
False


In [27]:
cb_dict = {n.install.__name__: n.install, str(o.install): o.install, str(p.install): p.install}

test1 installed!


In [23]:
for x in cb_dict:
    print(cb_dict[x] == n.install)

True
False
False


In [28]:
cb_dict

{'None': <bound method Named_Object.install of test1>,
 '<bound method Named_Object.install of test2>': <bound method Named_Object.install of test2>,
 '<bound method Named_Object.install of test3>': <bound method Named_Object.install of test3>}

In [53]:
str(n.install.__self__) + '.' + str(n.install.__func__).split('.')[1].split()[0]

'test1.install'

In [58]:
x = str(n.install).split()
x[-1].replace('>', '.') + x[2]

'test1.Named_Object.install'